# ETL + Dashboard

The idea was to get data from an API, extract, transform and load it to create an interactive dashboard to visualize and analyze the data at the end.


### Getting data from Free Stock API

Free Stock API called [ALPHA VANTAGE](https://www.alphavantage.co)

API KEY: Code that you get in the API website

In [ ]:
#!pip install jupyter-dash #necessary to install with you don't have

In [ ]:
# importing libraries
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import requests


# ========= getting the stock data =========== #
# replace YOUR_API_KEY with your actual Alpha Vantage API key
api_key = 'X6V1IF52C5QX0S7S'
#"<CODE>" # Fill with the code that you get in the API website

# define a function to retrieve stock data from Alpha Vantage API
def get_stock_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

# define a function to calculate dividend yield
def calculate_dividend_yield(row):
    if row["Adj Close"] == 0:
        return 0
    else:
        return row["Dividend Amount"] / row["Adj Close"] * 100

# define a list of stock symbols that you want to analyze
symbols = ['KLBN4.SA', 'UNIP6.SA', 'TAEE11.SA', 'RECR11.SA', 'CVBI11.SA'] 

# create an empty DataFrame to store the results
results = pd.DataFrame()

# loop through the list of symbols and retrieve the data
for symbol in symbols:
    data = get_stock_data(symbol)

    # extract the monthly data and convert the date column to a datetime object
    monthly_data = pd.DataFrame.from_dict(data['Monthly Adjusted Time Series'], orient='index')
    monthly_data.index = pd.to_datetime(monthly_data.index)

    # extract the adjusted close and dividend amount columns and convert them to float
    monthly_data["Adj Close"] = monthly_data["5. adjusted close"].astype(float)
    monthly_data["Dividend Amount"] = monthly_data["7. dividend amount"].astype(float)

    # calculate the dividend yield and add it as a column to the DataFrame
    monthly_data["Dividend Yield"] = monthly_data.apply(calculate_dividend_yield, axis=1)

    # group the data by month and select the last row of each group
    monthly_data = monthly_data.groupby(pd.Grouper(freq="M")).last()

    # add the symbol as a column to the DataFrame
    monthly_data["ID_Stock"] = symbol

    # append the data to the results DataFrame
    results = results.append(monthly_data)

# select the desired columns and reset the index
results = results.reset_index()[["index", "ID_Stock", "Adj Close", "Dividend Amount", "Dividend Yield"]]

# rename the columns
results = results.rename(columns={"index": "Date"})

# print the results
#print(results)

# ========= creating graphs =========== #
# figure 1
fig = px.bar(results, x="Date", y="Dividend Yield", color="ID_Stock", barmode="group")
stock_options = list(results['ID_Stock'].unique())
stock_options.append('All stocks')


# figure 2
fig2 = px.line(results, x="Date", y="Dividend Yield", color="ID_Stock")
stock_years = list(results['Date'].dt.year.unique())
stock_years.append ('All years')

# ========= starting dash - Dashboard =========== #
external_stylesheets = []

app = Dash(__name__, external_stylesheets=external_stylesheets)

# editing html file
app.layout = html.Div([

    html.Div(children=[
        html.H1(children='Track record stocks'),
        html.P(children='Dividend yield'),
        
        html.Div(children='''
            Dividend yield: a financial ratio that measures the annual dividend income per 
            share of a company relative to its share price. It is expressed as a percentage and is 
            calculated by dividing the annual dividend per share by the current market price per share.
        '''),
    
    
# editing graphs on dashboard
        dcc.Dropdown(stock_options, value='All stocks', id='stocks_lists'),
        #dcc.Slider(min=2000, max=2023, id='stocks_years', marks={i: str(i) for i in range(23)}, value=1),
        dcc.Graph(
            id='dividend_yield_graph',
            figure=fig,
            config={'displayModeBar': False},
        ),
    ], style={'width': '60%', 'display': 'inline-block'}),
    
    html.Div(children=[
        html.P(children='Track record stocks per year'),
        dcc.Dropdown(stock_years, value='All years', id='stocks_years'),
        dcc.Graph(
            id='dividend_yield_graph_line',
            figure=fig2)], style={'width': '60%', 'display': 'inline-block'}),

])

@app.callback(
    Output('dividend_yield_graph', 'figure'),
    Input('stocks_lists', 'value'),
    #Input('stocks_years', 'value')
)
def update_output(value):
    if value == 'All stocks': 
        fig = px.bar(results, x="Date", y="Dividend Yield", color="ID_Stock", barmode="group")
    else:
        filtered_table = results.loc[results['ID_Stock']==value, :]
        fig = px.bar(filtered_table, x="Date", y="Dividend Yield", color="ID_Stock", barmode="group")
    return fig

@app.callback(
    Output('dividend_yield_graph_line', 'figure'),
    Input('stocks_years', 'value')
)
def update_output(value2):
    if value2 == 'All years': 
        fig2 = px.line(results, x="Date", y="Dividend Yield", color="ID_Stock")
    else:
        filtered_table = results.loc[results['Date'].dt.year==value2, :]
        fig2 = px.line(filtered_table, x="Date", y="Dividend Yield", color="ID_Stock")
    return fig2


if __name__ == '__main__':
    app.run_server(host="127.0.0.1", port="8050")